In [1]:
import torch
import torchvision
from torch.optim import Adam

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
train_data = torchvision.datasets.FashionMNIST("./data/", train=True, transform=torchvision.transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=10, shuffle=True)

100.0%Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw
27.8%Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz
0.2%Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
100.0%Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw
159.1%Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
Processing...
Done!


In [4]:
test_data = torchvision.datasets.FashionMNIST("./data/", transform=torchvision.transforms.ToTensor(), download=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=10, shuffle=True)

In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(1, 16, 5, padding=2)
        self.conv2 = torch.nn.Conv2d(16, 32, 5, padding=2)
        self.act = torch.functional.F.relu
        self.bn1 = torch.nn.BatchNorm2d(16)
        self.bn2 = torch.nn.BatchNorm2d(32)
        self.pool = torch.nn.MaxPool2d(2,2)
        self.fc1 = torch.nn.Linear(49*32, 600)
        self.fc1_drop = torch.nn.Dropout(p=0.4)
        self.fc2 = torch.nn.Linear(600, 10)
        
    def forward(self, x):
        x = self.pool(self.act(self.bn1(self.conv1(x))))
        x = self.pool(self.act(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = self.act(self.fc1(x))
        x = self.fc1_drop(x)
        x = self.fc2(x)
        return x
    
net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=600, bias=True)
  (fc1_drop): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=600, out_features=10, bias=True)
)

In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.001)

In [7]:
avg_acc = []
test_acc = []

for epoch in range(15):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        result = net(images)
        loss = criterion(result, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%1000==0:
            print("Epoch {}\tBatch {}\tAvg loss {}".format(epoch, i, running_loss/1000))
            running_loss = 0.
            
    

Epoch 0	Batch 0	Avg loss 0.002145137310028076
Epoch 0	Batch 1000	Avg loss 0.6525661058928818
Epoch 0	Batch 2000	Avg loss 0.4645184733029455
Epoch 0	Batch 3000	Avg loss 0.40682775068469346
Epoch 0	Batch 4000	Avg loss 0.37608303254563363
Epoch 0	Batch 5000	Avg loss 0.33130914135696365
Epoch 1	Batch 0	Avg loss 0.00022568981349468232
Epoch 1	Batch 1000	Avg loss 0.3035770759230945
Epoch 1	Batch 2000	Avg loss 0.29845934800617396
Epoch 1	Batch 3000	Avg loss 0.29655269865272565
Epoch 1	Batch 4000	Avg loss 0.2953507335157483
Epoch 1	Batch 5000	Avg loss 0.3060435249456204
Epoch 2	Batch 0	Avg loss 0.00021368832886219024
Epoch 2	Batch 1000	Avg loss 0.2559183811501716
Epoch 2	Batch 2000	Avg loss 0.25922344559582416
Epoch 2	Batch 3000	Avg loss 0.25262633326428474
Epoch 2	Batch 4000	Avg loss 0.25476106130587867
Epoch 2	Batch 5000	Avg loss 0.25177802723506465
Epoch 3	Batch 0	Avg loss 0.0004311804175376892
Epoch 3	Batch 1000	Avg loss 0.221786389801302
Epoch 3	Batch 2000	Avg loss 0.22023916752434888
Epo

In [8]:
correct = 0
total = 0

with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)

        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    
print("Total accuracy is {}%".format(round(correct/total*100, 2)))

Total accuracy is 96.62%


In [9]:
class_correct = list(0. for i in range(len(train_data.classes)))
class_total = list(0. for i in range(len(train_data.classes)))

In [10]:
with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels)
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            
for i in range(10):
    print('Accuracy of %15s : %2d %%' % (
        train_data.classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     T-shirt/top : 92 %
Accuracy of         Trouser : 99 %
Accuracy of        Pullover : 95 %
Accuracy of           Dress : 97 %
Accuracy of            Coat : 92 %
Accuracy of          Sandal : 99 %
Accuracy of           Shirt : 92 %
Accuracy of         Sneaker : 96 %
Accuracy of             Bag : 99 %
Accuracy of      Ankle boot : 99 %
